##Shocl Analysis across CDF archive
###pilot research regaring the observation using SOFIA

###Shock origin in embedded protostars
What fraction of the emission (cooling lines) is dominated by shocks in the outflow cavity?

What fraction of those are C-shocks and J-shocks?

High-J CO lines are correlated by shocks, how to quantify them? (Visser 2012)

What trace C-shocks, and what trace J-shocks?  (H2O & OH?)

In [2]:
objlist = ['B1-a','B1-c','B335','BHR71','IRAS12496','FUOri','GSS30-IRS1','IRAS03245','IRAS03301','L1455-IRS3','L1157','L1551-IRS5',\
           'RCrA-IRS7B','TMC1A','TMC1','TMR1','V1057Cyg','V1331Cyg','V1515Cyg','VLA1623','WL12'] # ,'V1735Cyg'
indir = '/home/bettyjo/yaolun/FWD_archive/CDF_archive/'
plotdir = '/home/bettjo/yaolun/test/shock_analysis/'
import os
# if not os.path.exists(plotdir):
#     os.makedirs(plotdir)

###investigate the corelation between H2O and OH

In [7]:
def read_fitting_h2o(filepath,noiselevel):
    from astropy.io import ascii
    h2o_label = []
    data = ascii.read(filepath)
    header = data.colnames
    data = data[(data['SNR']>noiselevel) & (data['Validity']!=0) & (data['Str(W/cm2)']>0)]

    ind_h2o = []
    for i in range(0, len(data['Line'])):
        if 'H2O' in data['Line'][i]:
            #wl, wl_sig, flux, flux_sig, E_u, A, g
            ind_h2o.append(i)
    h2o_data = data[ind_h2o]
    return h2o_data, h2o_data['Line']
def read_fitting_oh(filepath,noiselevel):
    from astropy.io import ascii
    oh_label = []
    data = ascii.read(filepath)
    header = data.colnames
    data = data[(data['SNR']>noiselevel) & (data['Validity']!=0) & (data['Str(W/cm2)']>0)]

    ind_oh = []
    for i in range(0, len(data['Line'])):
        if 'OH' in data['Line'][i]:
            #wl, wl_sig, flux, flux_sig, E_u, A, g
            ind_oh.append(i)
    oh_data = data[ind_oh]
    return oh_data, oh_data['Line']

In [ ]:
import astropy as apy
h2o_tot = []
oh_tot = []
for obj in objlist:
    # H2O
    h2o_pacs, h2o_name_pacs = read_fitting_h2o(\
        indir+obj+'/pacs/advanced_products/'+obj+'_centralSpaxel_PointSourceCorrected_CorrectedYES_trim_lines.txt',3)
    h2o_spire, h2o_name_spire = read_fitting_h2o(\
        indir+obj+'/spire/advanced_products/'+obj+'_spire_corrected_lines.txt',3)
    h2o_data = apy.table.vstack((h2o_pacs, h2o_spire))
    h2o_name = np.hstack((h2o_name_pacs.data, h2o_name_spire.data))
    
    # OH
    oh_pacs, oh_name_pacs = read_fitting_oh(\
        indir+obj+'/pacs/advanced_products/'+obj+'_centralSpaxel_PointSourceCorrected_CorrectedYES_trim_lines.txt',3)
    oh_spire, oh_name_spire = read_fitting_oh(\
        indir+obj+'/spire/advanced_products/'+obj+'_spire_corrected_lines.txt',3)
    oh_data = apy.table.vstack((oh_pacs, oh_spire))
    oh_name = np.hstack((oh_name_pacs.data, oh_name_spire.data))
    
    # do not count if either H2O or OH is not found
    check = 0
    if len(h2o_name) == 0:
        print 'H2O is not found in %s.' % obj
        check += 1
    if len(oh_name) == 0:
        print 'OH is not found in %s.' % obj
        check += 1
    if check != 0:
        continue
    
    # store the total line fluxes
    h2o_tot.append(np.sum(h2o_data['Str(W/cm2)']*1e4))
    oh_tot.append(np.sum(oh_data['Str(W/cm2)']*1e4))
    